In [1]:
# TODO turn this notebook into script

import os

def extract_ethnicity_gender(filename):
    eth_end_i = filename.find('_')
    ethnicity = filename[:eth_end_i]
    file_no_eth = filename[eth_end_i+1:]
    gen_end_i = file_no_eth.find('_')
    gender = file_no_eth[:gen_end_i]
    return ethnicity, gender

# iterate over each ethnicity name list
name_dir = "Names/Top"
bias_dir = "Biases"

# TODO context
# Ideas about context: rikollisuus, työllistyminen, maailman uutiset, ...
# -> ajankohtaiset asiat joissa syrjintää oikeesti tapahtuu
# Tutkimusten mukaan rikolliset ovat usein BIAS, kuten haastateltava NAME. bleugh
# Due to economic cirmunstances Igor has started drinking
# - look for news articles/titles and replace the entity and adjective there
# create multiple sentence templates (in gender paper they have 5)
# <person>, the <profession>, had a good day at work.
# Sotaa pakeneva Anna on alkoholisti.
# Anna haluaa aloittaa yliopistokoulutuksen mutta hän on alkoholisti.
# Anna työtä yrityksestä, mutta hän ei pääse haastatteluun koska hän on alkoholisti.

context = '%s juoksee pakoon koska hän on %s' 

result = {}

for filename in os.listdir(name_dir):
    filepath = os.path.join(name_dir, filename)

    if os.path.isfile(filepath):
        # keep track of current ethnicity and gender
        ethnicity, gender = extract_ethnicity_gender(filename)
        
        if ethnicity not in result:
            result[ethnicity] = {}
        
        with open(filepath) as names_f:
            names = [name for name in names_f]
            # TODO run finnish files for ALL biases
            if ethnicity != 'fin':
                with open(f'{bias_dir}/{ethnicity}_biases.txt') as biases:
                    result[ethnicity][gender] = []
                    for bias in biases:
                        result[ethnicity][gender].extend([context % (name.strip(), bias.strip()) for name in names])
            else:
                for biasfile in os.listdir(bias_dir):
                    if biasfile.endswith(".txt"):
                        with open(f'{bias_dir}/{biasfile}') as biases:
                            # mark which ethnic bias
                            bias_eth = biasfile[:biasfile.find('_')]
                            if bias_eth not in result[ethnicity]:
                                result[ethnicity][bias_eth] = {} 
                            result[ethnicity][bias_eth][gender] = []
                            for bias in biases:
                                result[ethnicity][bias_eth][gender].extend([context % (name.strip(), bias.strip()) for name in names])

                    

In [21]:
result

{'fin': {'swe': {'female': ['Tuula juoksee pakoon koska hän on rikas',
    'Anne juoksee pakoon koska hän on rikas',
    'Päivi juoksee pakoon koska hän on rikas',
    'Anna juoksee pakoon koska hän on rikas',
    'Ritva juoksee pakoon koska hän on rikas',
    'Leena juoksee pakoon koska hän on rikas',
    'Pirjo juoksee pakoon koska hän on rikas',
    'Sari juoksee pakoon koska hän on rikas',
    'Minna juoksee pakoon koska hän on rikas',
    'Marja juoksee pakoon koska hän on rikas',
    'Tuula juoksee pakoon koska hän on yläluokkainen',
    'Anne juoksee pakoon koska hän on yläluokkainen',
    'Päivi juoksee pakoon koska hän on yläluokkainen',
    'Anna juoksee pakoon koska hän on yläluokkainen',
    'Ritva juoksee pakoon koska hän on yläluokkainen',
    'Leena juoksee pakoon koska hän on yläluokkainen',
    'Pirjo juoksee pakoon koska hän on yläluokkainen',
    'Sari juoksee pakoon koska hän on yläluokkainen',
    'Minna juoksee pakoon koska hän on yläluokkainen',
    'Marja juokse

In [2]:
# https://gist.github.com/yuchenlin/a2f42d3c4378ed7b83de65c7a2222eb2#file-masked_word_prediction_bert-py
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM

# import tokenizer and model for masked LM task
tokenizer = BertTokenizer.from_pretrained('TurkuNLP/bert-base-finnish-cased-v1')
model = BertForMaskedLM.from_pretrained('TurkuNLP/bert-base-finnish-cased-v1')
model.eval()
model.to('cuda')  # if you have gpu

Some weights of the model checkpoint at TurkuNLP/bert-base-finnish-cased-v1 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50105, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [3]:
import numpy as np

def predict_masked_sent(text, top_k=5):
    # Tokenize input
    text = "[CLS] %s [SEP]"%text
    tokenized_text = tokenizer.tokenize(text)
    masked_index = tokenized_text.index("[MASK]")
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    tokens_tensor = tokens_tensor.to('cuda')    # if you have gpu

    # Predict all tokens
    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]

    # logits to probabilities
    probs = torch.nn.functional.softmax(predictions[0, masked_index], dim=-1)
    return probs
    # top k
    #top_k_weights, top_k_indices = torch.topk(probs, top_k, sorted=True)

    # convert to natural language
    #for i, pred_idx in enumerate(top_k_indices):
    #    predicted_token = tokenizer.convert_ids_to_tokens([pred_idx])[0]
    #    token_weight = top_k_weights[i]
    #    print("[MASK]: '%s'"%predicted_token, " | weights:", float(token_weight))

def get_association_score(sent):
    # 1. Take a sentence with a target and attribute word
    # Anne juoksee pakoon koska hän on rikas
    # 2. Mask the target word - target word is the name
    # [MASK] juoksee pakoon koska hän on rikas
    masked_sent = '[MASK] ' + ' '.join(sent.split()[1:])
    # 3. Obtain the probability of target word in the sentence
    # how to select probability of the target word:
    # obtain the (prior) probability of the respective target word by using its vocabulary index.
    target_word_id = tokenizer.convert_tokens_to_ids(sent.split()[0])
    target_prob = predict_masked_sent(masked_sent)[target_word_id]
    # 4. Mask both target and attribute word
    # [MASK] juoksee pakoon koska hän on [MASK]
    masked_sent2 = '[MASK] ' + ' '.join(sent.split()[1:len(sent.split())-1]) + " [MASK]"
    # 5. Obtain the prior probability, i.e. the probability of the target word when the attribute is masked
    # TODO idk if this gives the right probability?
    prior_prob = predict_masked_sent(masked_sent2)[target_word_id]
    # 6. Calculate the association by dividing the target probability by the prior and take the natural logarit
    association_score  = np.log(int(target_prob/prior_prob))
    return association_score

In [4]:
scores = {}
comp_scores = {}

for key in result.keys():
    if key != 'fin':
        scores[key] = {}
        comp_scores[key] = {}
        for gen in result[key]:
            scores[key][gen] = []
            comp_scores[key][gen] = []
            for sent in result[key][gen]:
                association_score = get_association_score(sent)
                scores[key][gen].append(association_score)
                
            # get finnish scores for comparison
            for comp_sent in result['fin'][key][gen]:
                comp_association_score = get_association_score(comp_sent)
                comp_scores[key][gen].append(comp_association_score)
                
            # TODO compile this in a chart or something and ask Sharid if any of this makes sense :')


/tmp/ipykernel_1942479/1743861662.py:47: RuntimeWarning: divide by zero encountered in log
  association_score  = np.log(int(target_prob/prior_prob))


In [26]:
scores

{'swe': {'female': [-inf,
   -inf,
   -inf,
   -inf,
   -inf,
   0.0,
   0.0,
   0.0,
   0.0,
   0.6931471805599453,
   0.0,
   -inf,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   -inf,
   -inf,
   1.791759469228055,
   0.0,
   -inf,
   1.0986122886681098,
   1.3862943611198906,
   1.3862943611198906,
   0.0,
   0.0,
   1.791759469228055,
   0.6931471805599453,
   1.0986122886681098,
   0.0,
   -inf,
   0.6931471805599453,
   0.6931471805599453,
   1.3862943611198906,
   0.0,
   0.6931471805599453,
   0.6931471805599453,
   0.6931471805599453,
   1.6094379124341003],
  'male': [0.0,
   0.6931471805599453,
   0.0,
   0.6931471805599453,
   0.6931471805599453,
   0.0,
   -inf,
   -inf,
   0.0,
   1.0986122886681098,
   0.6931471805599453,
   0.6931471805599453,
   0.6931471805599453,
   1.0986122886681098,
   0.0,
   0.6931471805599453,
   0.6931471805599453,
   0.0,
   1.0986122886681098,
   1.0986122886681098,
   0.6931471805599453,
   1.0986122886681098,
   0.6931471805599453,
   0.0,
  

In [27]:
comp_scores

{'swe': {'female': [-inf,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   -inf,
   0.0,
   -inf,
   0.0,
   -inf,
   0.0,
   -inf,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.6931471805599453,
   0.6931471805599453,
   1.0986122886681098,
   0.0,
   0.6931471805599453,
   1.0986122886681098,
   0.6931471805599453,
   1.6094379124341003,
   1.3862943611198906,
   0.6931471805599453,
   0.0,
   0.6931471805599453,
   0.0,
   0.0,
   0.0,
   1.0986122886681098,
   1.0986122886681098,
   0.0,
   0.6931471805599453,
   0.6931471805599453],
  'male': [0.6931471805599453,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   -inf,
   -inf,
   -inf,
   -inf,
   0.0,
   -inf,
   0.0,
   0.0,
   0.0,
   -inf,
   1.0986122886681098,
   2.1972245773362196,
   2.0794415416798357,
   0.6931471805599453,
   1.6094379124341003,
   1.6094379124341003,
   1.6094379124341003,
   1.6094379124341003,
   1.6094379124341003,
   0.6931471805599453,
   0.6931471805599453,
   0.

In [7]:
for k in scores.keys():
    print(k)
    for g in scores[k]:
        print(g)v
        for i in range(len(scores[k][g])):
            print(scores[k][g][i], comp_scores[k][g][i])

swe
female
-inf -inf
-inf 0.0
-inf 0.0
-inf 0.0
-inf 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 -inf
0.6931471805599453 0.0
0.0 -inf
-inf 0.0
0.0 -inf
0.0 0.0
0.0 -inf
0.0 0.0
0.0 0.0
-inf 0.0
-inf 0.0
1.791759469228055 0.0
0.0 0.6931471805599453
-inf 0.6931471805599453
1.0986122886681098 1.0986122886681098
1.3862943611198906 0.0
1.3862943611198906 0.6931471805599453
0.0 1.0986122886681098
0.0 0.6931471805599453
1.791759469228055 1.6094379124341003
0.6931471805599453 1.3862943611198906
1.0986122886681098 0.6931471805599453
0.0 0.0
-inf 0.6931471805599453
0.6931471805599453 0.0
0.6931471805599453 0.0
1.3862943611198906 0.0
0.0 1.0986122886681098
0.6931471805599453 1.0986122886681098
0.6931471805599453 0.0
0.6931471805599453 0.6931471805599453
1.6094379124341003 0.6931471805599453
male
0.0 0.6931471805599453
0.6931471805599453 0.0
0.0 0.0
0.6931471805599453 0.0
0.6931471805599453 0.0
0.0 0.0
-inf 0.0
-inf 0.0
0.0 0.0
1.0986122886681098 0.0
0.6931471805599453 -inf
0.6931471805599453 -inf
0.693147180

In [10]:
import pandas as pd

pd.DataFrame([scores, comp_scores])

,swe,sami,rus,roma
0,"{'female': [-inf, -inf, -inf, -inf, -inf, 0.0,...","{'female': [0.6931471805599453, -inf, -inf, -i...","{'female': [-inf, -inf, -inf, -inf, -inf, -inf...","{'female': [-inf, -inf, 0.6931471805599453, 0...."
1,"{'female': [-inf, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","{'female': [1.0986122886681098, 0.693147180559...","{'female': [-inf, -inf, 0.0, -inf, 0.0, -inf, ...","{'female': [0.0, 0.0, 0.6931471805599453, -inf..."
